# MLP hyperparameter tuning using the Fashion MNIST dataset
We will train and evaluate an MLP on the Fashion MNIST dataset. It consists of 70.000 grayscale images of 28x28 pixels each, and there are 10 classes.

Hyperparameters:
1. Optimizer: Momentum optimization
2. Activation function: SELU
3. Weight initialization: LeCun
4. Learning rate schedule: Performance scheduling
5. Regularization: Alpha dropout

## Setup

In [12]:
# Common imports
import sys
import os
import sklearn
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib.pyplot as plt

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Load the data

In [13]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
# Split the full training set into a validation set and a (smaller) training set.
# Normalize the input data to a range from 0 to 1.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

## Standardize the data
Because we want to use the SELU activation function and LeCun weight initializer, we should standardize all the input features to a mean of 0 and a standard deviation of 1. Since each pixel is an input feature, there are 28x28=784 input features, and we need to compute the mean and standard deviation for each of them.

In [14]:
# Compute the mean for each pixel.
pixel_means = X_train.mean(axis=0)
pixel_means.shape

(28, 28)

In [15]:
# Compute the standard deviation for each pixel.
pixel_stds = X_train.std(axis=0)
pixel_stds.shape

(28, 28)

In [16]:
# Scale the inputs to mean 0 and standard deviation 1 to achieve self-normalization with SELU and LeCun.
X_train_standardized = (X_train - pixel_means) / pixel_stds

X_valid_standardized = (X_valid - X_valid.mean(axis=0)) / X_valid.std(axis=0)
X_test_standardized = (X_test - X_test.mean(axis=0)) / X_test.std(axis=0)

In [17]:
# Validate that the mean is close to 0 for each pixel.
X_train_standardized.mean(axis=0)

array([[ 2.02095556e-15,  1.62754053e-15,  4.46262219e-16,
         1.14067644e-15, -7.41440495e-16, -2.63235242e-15,
        -1.34868732e-15, -2.60230625e-15,  1.66292233e-16,
         6.41022589e-16,  1.30749956e-16, -1.75860942e-15,
         1.77240041e-16, -4.50241864e-16,  5.90933363e-16,
        -7.87533675e-16,  2.07429427e-15, -2.87339849e-16,
        -4.16042958e-16, -6.74107235e-17,  2.68472416e-15,
        -2.27123371e-16, -5.26867943e-16, -1.31060062e-15,
        -2.53719217e-15, -4.31397342e-17,  1.73844777e-16,
         1.29615371e-15],
       [ 3.54259040e-15, -7.59241659e-16,  4.87038213e-15,
        -8.00087774e-16, -1.62997292e-15,  7.67104562e-16,
         1.68777820e-15, -2.78451504e-15,  6.71137893e-16,
        -2.99043013e-15, -2.08019057e-15,  7.63978779e-16,
         7.48899932e-16,  1.82329707e-15, -5.25664361e-16,
        -9.88134826e-16,  9.29450456e-16, -5.45922903e-16,
        -1.89077037e-16,  4.06024709e-16, -7.56264748e-16,
         5.24574323e-16, -5.09

In [18]:
# Validate that the standard deviation is close to 1 for each pixel.
X_train_standardized.std(axis=0)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1

## Create a model using the Sequential API

In [19]:
model = keras.models.Sequential()
# Input layer:
model.add(keras.layers.Flatten(input_shape=[28, 28]))

# Hidden layers:
for layer in range(3):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))

# Use alpha dropout in the last hidden layer.
model.add(keras.layers.AlphaDropout(rate=0.2))
    
# Output layer.
model.add(keras.layers.Dense(10, activation="softmax"))

c:\Users\kille\anaconda3\envs\homl3\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Compile the model
You must at least specify the loss function and the optimizer to use. You can also specify a list of additional metrics to use during training and evaluation.

In [10]:
# The optimixer is Stochastic Gradient Descent with momentum optimization. The momentum is set to 0.9.
# This value usually works well in practice. We use the default learning rate (0.01).
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(momentum=0.9),
              metrics=["accuracy"])

## Train the model

In [11]:
# EarlyStopping (with rollback to the best model).
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# Performance scheduling
# (multiply the learning rate by a factor when the error stops dropping for a number of epochs, called patience)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=4)

# Train the model with early stopping or performance scheduling or both. Training is much faster when
# early stopping is used, but a slightly better accuracy is achieved with performance scheduling alone.
history = model.fit(X_train_standardized, y_train, epochs=40,
                    validation_data=(X_valid_standardized, y_valid),
                    callbacks=[lr_scheduler, early_stopping])

Epoch 1/40


TypeError: Exception encountered when calling AlphaDropout.call().

[1mgreater_equal() got an unexpected keyword argument 'seed'[0m

Arguments received by AlphaDropout.call():
  • inputs=tf.Tensor(shape=(None, 100), dtype=float32)
  • training=True

In [ ]:
# Show the learning curves.
# (The training curves should be shifted half an epoch to the left to be completely comparable with
# the validation curves).

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

## Evaluate the model.

In [ ]:
model.evaluate(X_test_standardized, y_test)